In [1]:
%config IPCompleter.greedy=True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import tensorflow.contrib.eager as tfe
import tqdm
tf.enable_eager_execution()

## download imagenet dataset

In [3]:
# !wget http://image-net.org/imagenet_data/urls/imagenet_fall11_urls.tgz

In [4]:
# !tar -zxvf imagenet_fall11_urls.tgz

### download images from urls
---
imagenet 디렉토리를 생성
---

In [ ]:
!mkdir imagenet 

In [ ]:
import os
import tqdm

file_path = "./fall11_urls.txt"
with open(file_path) as file:
#     for line in tqdm(file):
    for idx, line in enumerate(file):
        sub_line = line.split()
        sub_line2 = sub_line[0].split("_")
        if len(sub_line2) != 2 :
            continue 
        dir_name = sub_line2[0]
        file_name = sub_line2[1]
        dir_path = os.path.join("./imagenet",dir_name)
        if not os.path.isdir(dir_path) :
            os.mkdir(dir_path)
        output_path = os.path.join("./imagenet",dir_name,file_name+".jpg")
        if os.path.exists(output_path):
            #이미 파일이 있을 경우 넘어 간다. 
            continue
        cmd = 'wget {} -o {}'.format(sub_line[1], output_path)
        os.system(cmd)
        if (idx % 10) == 0 :
            print("downloading now...",idx)


downloading now... 30
downloading now... 40
downloading now... 50
downloading now... 60
downloading now... 70
downloading now... 80
downloading now... 90
downloading now... 100
downloading now... 110
downloading now... 120
downloading now... 130


VGG19 Model <br>

<ul>
block 1<Br>
conv 3*3, 64<br>
conv 3*3, 64<br>
maxpool 2 
</ul>

<ul>
block 2<Br>
conv 3*3, 128<br>
conv 3*3, 128<br>
maxpool 2 
</ul>

<ul>
block 3<Br>
conv 3*3, 256<br>
conv 3*3, 256<br>
conv 3*3, 256<br>
maxpool 2
</ul>

<ul>
block 4<Br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
maxpool 2 
</ul>


<ul>
block 5<Br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
maxpool 2 
</ul>

<ul>
block 6 Fully Connect<Br>
Fully Connect Network 4096<br>
Fully Connect Network 4096<br>
Fully Connect Netowrk 100<br>
</ul>


In [ ]:
class ConvLayer2(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer2, self).__init__()
        #padding이 same이 아닐 경우 이미지 크기가 줄어 들어서 오작동 한다.
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.pool(model)
        return model

class ConvLayer3(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer3, self).__init__()
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv3 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.conv3(model)
        model = self.pool(model)
        return model

class ConvLayer4(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer4, self).__init__()
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv3 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.conv4 = layers.Conv2D(filters, [size,size], activation='relu', padding='same')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.conv3(model)
        model = self.conv4(model)
        model = self.pool(model)
        return model

class VGG19(tf.keras.Model):
    def __init__(self, img_class):
        super(VGG19, self).__init__()
        self.conv_block1 = ConvLayer2(64, 3)
        self.conv_block2 = ConvLayer2(128, 3)
        self.conv_block3 = ConvLayer3(256, 3)
        self.conv_block4 = ConvLayer4(512, 3)
        self.conv_block5 = ConvLayer4(512, 3)
#         self.conv_block6 = ConvLayer4(512, 3)
        self.flat = layers.Flatten()
        self.fcn1 = layers.Dense(4096, activation='relu')
        self.fcn2 = layers.Dense(4096, activation='relu')
        self.fcn3 = layers.Dense(img_class, activation='relu')
        
    def call(self, x):
        model = self.conv_block1(x)
        model = self.conv_block2(model)
        model = self.conv_block3(model)
        model = self.conv_block4(model)
        model = self.conv_block5(model)
#         model = self.conv_block6(model)
        model = self.flat(model)
        model = self.fcn1(model)
        model = self.fcn2(model)
        model = self.fcn3(model)
        return model

In [ ]:
model = VGG19(100)
print(model)
x = tf.random_normal([1,224,224,3])
print(model(x).shape)

In [ ]:
# tf.keras.datasets.
# dataset = cifar100
(cifar_images, cifar_labels), _ = cifar100.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(cifar_images[...,tf.newaxis]/255, tf.float32),
 tf.cast(cifar_labels, tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [ ]:
print(cifar_images[0:1].shape)
print(model(cifar_images[0:1]).shape)

In [ ]:
num_batch = len(cifar_images)/ 32
optimizer = tf.train.AdamOptimizer()
epoch=2
loss_history = []
acc_history = []

for _ in range(epoch):
    for ( batch, (images, labels)) in enumerate(dataset.take(num_batch)):
        accuracy = tfe.metrics.Accuracy('accuracy', dtype=tf.float32)
        with tf.GradientTape() as tape:
            logits = model(images)
            loss_value = tf.losses.sparse_softmax_cross_entropy(labels, logits)
            accuracy(tf.argmax(logits, axis=1, output_type=tf.int64),
                    tf.cast(labels, tf.int64))

        loss_history.append(loss_value.numpy())
        acc_history.append(accuracy.result())
        grads = tape.gradient(loss_value, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables), 
                                 global_step=tf.train.get_or_create_global_step())
        if batch % 20== 0 :
            print("Loss at step {:03d}: {:.3f}, Accu : {:.3f}".format(batch, loss_value, accuracy.result()))
